# Exercise 0_4

#### b) Use gemini to simulate 20 data points in json format containing the following fields: first_name, last_name, phone_number, email, department, salary, title. See if you can prompt to direct the LLM output to have swedish names, phone numbers in swedish format (+46 731 29 52), departments (IT, HR, marketing, sales), reasonable salary (you might need to check some swedish statistics on salaries) and corresponding titles within these departments.

In [2]:
from google import genai
from google.genai import types

prompt = "first_name, last_name, phone_number, email, department, salary, title"
system_instructuion = """
    Generate 20 data records of employees of a small company.
    You will receive the columns of data in the prompt
    The names should be in Swedish, both men and women, both name and last name
    phone numbers should have swedish format (e.g. +46 731 29 52)
    Departments should be in English and be: IT, HR, marketing, sales 
    Salaries should be reasonable Swedish salaries for a Stockholm based company
    Titles should adhere to its departments, for instance marketing manager. They should be in English.
    The output format MUST be a json and MUST NOT include anything else but this json.
    """

client = genai.Client()
response = client.models.generate_content(
    model = "gemini-2.0-flash",
    config=types.GenerateContentConfig(
    system_instruction=system_instructuion),
    contents = prompt,
    )
data = response.text
print(data)

```json
[
  {
    "first_name": "Erik",
    "last_name": "Andersson",
    "phone_number": "+46 705 12 34 56",
    "email": "erik.andersson@example.com",
    "department": "IT",
    "salary": 45000,
    "title": "Software Developer"
  },
  {
    "first_name": "Anna",
    "last_name": "Larsson",
    "phone_number": "+46 739 87 65 43",
    "email": "anna.larsson@example.com",
    "department": "HR",
    "salary": 52000,
    "title": "HR Specialist"
  },
  {
    "first_name": "Johan",
    "last_name": "Svensson",
    "phone_number": "+46 721 23 45 67",
    "email": "johan.svensson@example.com",
    "department": "Marketing",
    "salary": 60000,
    "title": "Marketing Manager"
  },
  {
    "first_name": "Maria",
    "last_name": "Nilsson",
    "phone_number": "+46 708 34 56 78",
    "email": "maria.nilsson@example.com",
    "department": "Sales",
    "salary": 55000,
    "title": "Sales Representative"
  },
  {
    "first_name": "Peter",
    "last_name": "Karlsson",
    "phone_number": "+

#### c) Now use pydantic to validate this json and put in proper schema that the fields should follow. You might need to do some processing such as removing backticks and maybe loading json data into a list with json.loads(). Also make sure that only correctly validated data should be stored.

#### d) Write this json data to a folder called output_data.

In [ ]:
from pydantic import BaseModel, ValidationError, EmailStr
from typing import Literal, List
from pathlib import Path

class Employee(BaseModel):
    first_name : str
    last_name : str
    phone_number : str
    email : EmailStr
    department: Literal["IT", "HR", "marketing", "sales"]
    salary : int
    title : str

class EmployeeData(BaseModel):
    employees: List[Employee]


from google import genai
from google.genai import types

prompt = "first_name, last_name, phone_number, email, department, salary, title"
system_instructuion = """
    Generate 20 data records of employees of a small company.
    You will receive the columns of data in the prompt
    The names should be in Swedish, both men and women, both name and last name
    phone numbers should have swedish format (e.g. +46 731 29 52)
    Departments should be in English and be: IT, HR, marketing, sales 
    Salaries should be reasonable Swedish salaries for a Stockholm based company
    Titles should adhere to its departments, for instance marketing manager. They should be in English.
    The output format MUST be a json and MUST NOT include anything else but this json.
    """

client = genai.Client()
response = client.models.generate_content(
    model = "gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=system_instructuion,
        response_mime_type="application/json", 
        response_schema=EmployeeData,
    ),
    contents = prompt,
    )

try:
    EmployeeData.model_validate_json(response.text)
    #print(response.text)
    json_path = Path.cwd() / "assets" / "json_ex_4"
    json_path.mkdir(parents=True, exist_ok=True)
    json_file = json_path / "Employee_dump.json"
    json_file.write_text(response.text, encoding="utf-8")
except ValidationError as e:
    print(e)

#### e) Use pandas to read the data as dataframe
#### f) Write a csv file to your output_data

In [47]:
import pandas as pd
import json

class Employee(BaseModel):
    first_name : str
    last_name : str
    phone_number : str
    email : EmailStr
    department: Literal["IT", "HR", "marketing", "sales"]
    salary : int
    title : str

class EmployeeData(BaseModel):
    employees: List[Employee]


from google import genai
from google.genai import types

prompt = "first_name, last_name, phone_number, email, department, salary, title"
system_instructuion = """
    Generate 20 data records of employees of a small company.
    You will receive the columns of data in the prompt
    The names should be in Swedish, both men and women, both name and last name
    phone numbers should have swedish format (e.g. +46 731 29 52)
    Departments should be in English and be: IT, HR, marketing, sales 
    Salaries should be reasonable Swedish salaries for a Stockholm based company
    Titles should adhere to its departments, for instance marketing manager. They should be in English.
    The output format MUST be a json and MUST NOT include anything else but this json.
    """

client = genai.Client()
response = client.models.generate_content(
    model = "gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=system_instructuion,
        response_mime_type="application/json", 
        response_schema=EmployeeData,
    ),
    contents = prompt,
    )

try:
    EmployeeData.model_validate_json(response.text)
    json_file = Path("employee_dump_directly.json")
    json_file.write_text(response.text, encoding='utf-8') 
    with open(json_file, "r", encoding='utf-8') as f:
        employee_dict = json.load(f)
    employee_list = employee_dict["employees"]
    df = pd.json_normalize(employee_list)
    df.to_csv(Path.cwd()/"assets"/"employees.csv")
    print(df)
except ValidationError as e:
    print(e)

   first_name   last_name      phone_number                           email  \
0        Erik   Andersson  +46 705 12 34 56      erik.andersson@example.com   
1        Anna    Svensson  +46 739 87 65 43       anna.svensson@example.com   
2       Johan     Nilsson  +46 721 23 45 67       johan.nilsson@example.com   
3       Maria    Karlsson  +46 707 34 56 78      maria.karlsson@example.com   
4        Lars   Lindström  +46 735 45 67 89      lars.lindstrom@example.com   
5       Sofia     Persson  +46 709 56 78 90       sofia.persson@example.com   
6      Anders     Bergman  +46 723 67 89 01      anders.bergman@example.com   
7       Karin   Lundqvist  +46 701 78 90 12     karin.lundqvist@example.com   
8      Magnus       Ström  +46 737 89 01 23        magnus.strom@example.com   
9        Lena        Holm  +46 703 90 12 34           lena.holm@example.com   
10      Peter     Ekström  +46 725 01 23 45       peter.ekstrom@example.com   
11        Eva     Jonsson  +46 708 12 34 56         

In [115]:
import pandas as pd
import json

class Employee(BaseModel):
    first_name : str
    last_name : str
    phone_number : str
    email : EmailStr
    department: Literal["IT", "HR", "marketing", "sales"]
    salary : int
    title : str

class EmployeeData(BaseModel):
    employees: List[Employee]


from google import genai
from google.genai import types

prompt = "first_name, last_name, phone_number, email, department, salary, title"
system_instructuion = """
    Generate 20 data records of employees of a small company.
    You will receive the columns of data in the prompt
    The names should be in Swedish, both men and women, both name and last name
    phone numbers should have swedish format (e.g. +46 731 29 52)
    Departments should be in English and be: IT, HR, marketing, sales 
    Salaries should be reasonable Swedish salaries for a Stockholm based company
    Titles should adhere to its departments, for instance marketing manager. They should be in English.
    The output format MUST be a json and MUST NOT include anything else but this json.
    """

client = genai.Client()
response = client.models.generate_content(
    model = "gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=system_instructuion,
        response_mime_type="application/json", 
        response_schema=EmployeeData,
    ),
    contents = prompt,
    )

try:
    employee_data = EmployeeData.model_validate_json(response.text)
    serialized_employees = employee_data.model_dump()["employees"]
    employ_df = pd.DataFrame(serialized_employees)
    print(employ_df)
    
except ValidationError as e:
    print(e)

   first_name    last_name      phone_number                         email  \
0        Erik     Svensson  +46 701 23 45 67     erik.svensson@example.com   
1        Anna     Karlsson  +46 732 34 56 78     anna.karlsson@example.com   
2        Lars    Lindström  +46 723 45 67 89    lars.lindstrom@example.com   
3       Maria      Nilsson  +46 704 56 78 90     maria.nilsson@example.com   
4       Johan    Andersson  +46 735 67 89 01   johan.andersson@example.com   
5        Sara      Persson  +46 706 78 90 12      sara.persson@example.com   
6       Peter      Jonsson  +46 727 89 01 23     peter.jonsson@example.com   
7        Lena    Bergström  +46 708 90 12 34    lena.bergstrom@example.com   
8      Mikael      Ekström  +46 739 01 23 45    mikael.ekstrom@example.com   
9       Sofia    Lundqvist  +46 700 12 34 56   sofia.lundqvist@example.com   
10     Anders         Holm  +46 721 23 45 67       anders.holm@example.com   
11      Karin   Gustafsson  +46 702 34 56 78  karin.gustafsson@e

#### g) Load this data into a staging layer and store this into a table called employees.

In [55]:
import duckdb

with duckdb.connect("company.db") as con:
    con.sql("CREATE SCHEMA IF NOT EXISTS staging")
    con.execute("CREATE TABLE IF NOT EXISTS staging.employees AS SELECT * FROM df")
    con.execute("INSERT INTO staging.employees SELECT * FROM df")
    con.table("staging.employees").show()

┌────────────┬────────────┬──────────────────┬────────────────────────────────┬────────────┬────────┬──────────────────────────────┐
│ first_name │ last_name  │   phone_number   │             email              │ department │ salary │            title             │
│  varchar   │  varchar   │     varchar      │            varchar             │  varchar   │ int32  │           varchar            │
├────────────┼────────────┼──────────────────┼────────────────────────────────┼────────────┼────────┼──────────────────────────────┤
│ Erik       │ Andersson  │ +46 705 12 34 56 │ erik.andersson@example.com     │ IT         │  45000 │ Software Engineer            │
│ Anna       │ Svensson   │ +46 739 87 65 43 │ anna.svensson@example.com      │ HR         │  42000 │ HR Coordinator               │
│ Johan      │ Nilsson    │ +46 721 23 45 67 │ johan.nilsson@example.com      │ marketing  │  50000 │ Marketing Manager            │
│ Maria      │ Karlsson   │ +46 707 34 56 78 │ maria.karlsson@example

#### h) Use gemini to simulate departments data. There should be same departments as those you had in task b. Also add a description field and a contact person.
#### i) Add a departments table in your duckdb database under staging layer to store this data.

In [ ]:
class Department(BaseModel):
    department: Literal["IT", "HR", "marketing", "sales"]
    description : str
    contact_person : str

class DepartmentData(BaseModel):
    departments : List[Department]


prompt = "department, description and contact_person"
system_instructions =  """
    Generate departments data based on the columns given to you in the prompt
    You MUST output JSON data only, no other data.
    """

client = genai.Client()
response = client.models.generate_content(
    model = "gemini-2.0-flash",
    config=types.GenerateContentConfig(
        response_mime_type="application/json", 
        response_schema=DepartmentData,
    ),
    contents = prompt,
    )

try:
    departments_obj = DepartmentData.model_validate_json(response.text)
    
    serialized_deployments = departments_obj.model_dump()
    department_df = pd.DataFrame(serialized_deployments["departments"])
    
    with duckdb.connect("company.db") as con:
        con.execute("CREATE TABLE IF NOT EXISTS staging.departments AS SELECT * FROM department_df")
        con.execute("INSERT INTO staging.departments SELECT * FROM department_df")
        con.table("staging.departments").show()
    
except ValidationError as e:
    print(e)

┌────────────┬───────────────────────────────────────────────────────────────────────────────────────────┬────────────────┐
│ department │                                        description                                        │ contact_person │
│  varchar   │                                          varchar                                          │    varchar     │
├────────────┼───────────────────────────────────────────────────────────────────────────────────────────┼────────────────┤
│ IT         │ Provides technical support and maintains the company's IT infrastructure.                 │ John Smith     │
│ HR         │ Manages employee relations, recruitment, and benefits.                                    │ Jane Doe       │
│ marketing  │ Develops and executes marketing strategies to promote the company's products or services. │ Peter Jones    │
│ sales      │ Responsible for selling the company's products or services to customers.                  │ Mary Brown     │
│ IT    